# Calibration

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

CHECKERBOARD = (14, 10)
square_size_mm = 6
subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = (
    cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
    + cv2.fisheye.CALIB_CHECK_COND
    + cv2.fisheye.CALIB_FIX_SKEW
)
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0, :, :2] = (
    np.mgrid[0 : CHECKERBOARD[0], 0 : CHECKERBOARD[1]].T.reshape(-1, 2) * square_size_mm
)
cap = cv2.VideoCapture(1)

cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)

In [ ]:
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

In [ ]:
_ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = cv2.findChessboardCorners(
    gray,
    CHECKERBOARD,
    cv2.CALIB_CB_ADAPTIVE_THRESH
    + cv2.CALIB_CB_FAST_CHECK
    + cv2.CALIB_CB_NORMALIZE_IMAGE,
)
# If found, add object points, image points (after refining them)
if ret == True:
    objpoints.append(objp)
    cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), subpix_criteria)
    imgpoints.append(corners)
    cv2.drawChessboardCorners(frame, CHECKERBOARD, corners, ret)
plt.imshow(frame)
plt.show()

If detected output is undesirable, pop that set of obj/img points.

In [ ]:
objpoints.pop()
imgpoints.pop()

In [ ]:
cap.release()

In [ ]:
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = cv2.fisheye.calibrate(
    objpoints,
    imgpoints,
    gray.shape[::-1],
    K,
    D,
    rvecs,
    tvecs,
    calibration_flags,
    (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6),
)
print("Found " + str(N_OK) + " valid images for calibration")
# print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
D
print(rms)

# Undistortion

In [ ]:
_, frame = cap.read()
plt.imshow(frame)

In [ ]:
undistortframe = cv2.fisheye.undistort(
    frame,
    np.array(
        [
            [789.9416456819346, 0.0, 643.8350456739382],
            [0.0, 791.3634139038238, 351.1992571416615],
            [0.0, 0.0, 1.0],
        ]
    ),
    np.array(
        [
            [-0.10975678165468084],
            [0.04428627707749935],
            [-0.08883217424554242],
            [0.07713861990052967],
        ]
    ),
)
plt.imshow(undistortframe)
undistortframe